In [1]:
from IPython.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

# Vertice

In [2]:
import math
class Vertice:
    '''
    Classe que representa um vertice tendo como atributos:
        - o valor do vertice que pode ser outro objeto qualquer (um número, uma string, etc...)
        - um conjunto de arestas incidentes ao vertice
    Criacao do objeto: v = Vertice(valor)
    '''
    def __init__(self, valor, arestas = set(), direcionado=True, distanciaMinima = math.inf, verticeAnterior = None):
        '''
        Metodo construtor da classe
        '''
        self.__valor = valor
        self.__arestas = arestas
        self.__direcionado = direcionado
        self.__distanciaMinima = distanciaMinima
        self.__verticeAnterior = verticeAnterior

    def __lt__(self, vertice):
        if self.getDistanciaMinima() < vertice.getDistanciaMinima():
            return self
        else:
            return vertice
    
    def getVerticeAnterior(self):
        return self.__verticeAnterior

    def setVerticeAnterior(self, verticeAnterior):
        self.__verticeAnterior = verticeAnterior

    def getDistanciaMinima(self):
        return self.__distanciaMinima
    
    def setDistanciaMinima(self, distanciaMinima):
        self.__distanciaMinima = distanciaMinima

    def getValor(self):# retorna o valor do vertice
        return self.__valor
    
    def setValor(self, valor):# prove um valor ao vertice
        self.__valor = valor
        
    def getArestas(self): # retorna as arestas do vertice
        return self.__arestas
    
    def setAresta(self, aresta):# prove uma aresta ao vertice
        self.__arestas.add(aresta)
        
    def getArestasSaida(self): # retorna uma lista com as arestas que saem do vertice
        try:
            if self.__direcionado == False:
                return self.__arestas
            arestasDeSaida = []
            for aresta in self.__arestas:
                if aresta.getVerticeOrigem() == self:
                    arestasDeSaida.append(aresta)
            return arestasDeSaida
        except AttributeError: 
            return []
    
    def getArestasEntrada(self):# retorna uma lista com as arestas que entram do vertice
        try:
            if self.__direcionado == False:
                return self.__arestas
            arestasSaida = []
            for aresta in self.__arestas:
                if aresta.getVerticeDestino() == self:
                    arestasSaida.append(aresta)
            return arestasSaida
        except AttributeError: 
            return []
    
    def getGrau(self):# retorna o grau do vertice
        return len(self.getArestasSaida()) + len(self.getArestasEntrada())
    
    def getAdjacentes(self): # retorna uma lista com os vertices adjacentes ao vertice v
        listaVerticesAdjacentes = []
        for arestas_de_saida in self.getArestasSaida():
            listaVerticesAdjacentes.append(arestas_de_saida.getVerticeDestino())
        return listaVerticesAdjacentes

# Aresta

In [3]:
class Aresta:
    '''
    Classe que representa uma aresta tendo como atributos:
        - vertice_origem: o vertice de origem (v:Vertice)
        - vertice_destino: o vertice de destino (v:vertice)
        - peso: o peso da aresta que pode ser um objeto qualquer (int, str, etc...)
        - direcionada: flag indicando de a aresta e direcionado ou não (padrao True)
    '''
    def __init__(self, vertice_origem:Vertice, vertice_destino:Vertice, peso = 1, direcionada = True):
        '''
        Metodo construtor da classe: preenche os atributos
        '''
        self.__vertice_origem = vertice_origem
        self.__vertice_destino = vertice_destino
        self.__peso = peso
        self.__direcionada = direcionada
        self.__vertice_origem.setAresta(self)
        self.__vertice_destino.setAresta(self)
        
    def getVerticeOrigem(self):
        return self.__vertice_origem
    def getVerticeDestino(self):
        return self.__vertice_destino
    def getPeso(self):
        return self.__peso

# Grafo

In [4]:
from collections import deque
class Grafo:
    '''
    Classe que representa o grafo tendo como atributos:
        - um conjunto de vertices nao vazio
        - um conjunto de arestas (vazio ou nao)
    '''
    def __init__(self):
        '''
        Metodo construtor da classe: preenche os atributos
        '''
        self.__vertices = set()
        self.__arestas  = set()
        
    def setVertices(self, vertices):# prove o conjunto de vertices do grafo
        self.__vertices = vertices

    def getVertices(self):# retorna o conjunto de vertices do grafo
        return self.__vertices
        
    def setArestas(self, arestas):# prove o conjunto de arestas do grafo
        self.__arestas = arestas
        
    def getArestas(self):# retorna o conjunto de arestas do grafo
        return self.__arestas
    
    def getVerticeByValor(self, valor):# retorna um objeto da classe Vertice contendo um valor informado, se existir
        for v in self.__vertices:
            if v.getValor() == valor:
                return v
        return None

    def adicionarVertice(self, valor):# Adiciona um vertice ao grafo, caso nao exista vertice com o mesmo valor
        if self.buscarPorValor(valor) != valor:# valor nao esta no grafo
            self.__vertices.add(Vertice(valor))
            return True
        return False
    
    def adicionarAresta(self, origem, destino, peso = 1, direcionada = True):# adicionado uma aresta ao grafo ligando 2 vertices
        try:
            verticeOrigem = self.getVerticeByValor(origem)
            verticeDestino = self.getVerticeByValor(destino)
            if (verticeOrigem or verticeDestino) is None:  # existem os vertices de origem e destino?
                print("Nao ha no grafo, vertices de origem ou de destino com os valores informados.")
            self.getArestas().add(Aresta(verticeOrigem, verticeDestino, peso, direcionada))
        except AttributeError as error:
            print("Exceção: Nao ha no grafo, vertices de origem ou de destino com os valores informados.")
    
    def checkHandShakingLemma(self):# checa o lema do "aperto de mao"
        somaGraus = 0
        for v in self.getVertices():
            somaGraus+= v.getGrau()
        if somaGraus == len(self.getArestas())*2:
            return True
        else:
            return False
        
    def dfs(self, graph, v, visitados=[]):# busca em profundidade recursiva
        if v not in visitados: # se v nao foi visitado
            visitados.append(v) # marca vertice como visitado
        if len(v.getAdjacentes()) == 0: # vertice escolhido nao tem adjacentes
            self.dfs(graph, next(iter(graph.getVertices())), visitados) # chamada recursiva pegando o proximo vertice do set
        else: # vertice escolhido tem adjacentes
            for adjacente in v.getAdjacentes(): #percorre todos os adjacentes a ele
                if adjacente not in visitados: # se um dos adjacentes nao foi visitado
                    self.dfs(graph, adjacente, visitados) # chamada recursiva para cada adjacente
        return visitados
    
    def bfs(self, v, visitados = [], fila = deque([])):# busca em largura recursiva
        fila.append(v)  # adiciona o vertice v a fila
        if v not in visitados:  # se vertice v nao esta em visitados
            visitados.append(v)  # adiciona vertice v a visitados
        while fila:  # enquanto houver vertices na fila
            vertice = fila.popleft()  # tira vertice ja visitado da fila
            if len(vertice.getArestasSaida()) == 0: # vertice escolhido nao tem adjacentes
                self.bfs(next(iter(self.getVertices())), visitados, fila) # chamada recursiva pegando o proximo vertice do set   
            else:
                for adjacente in vertice.getAdjacentes(): #percorre todos os adjacentes a ele
                    if adjacente not in visitados: # se um dos adjacentes nao foi visitado
                        visitados.append(adjacente)  # insere o adjacente em visitados
                        self.bfs(adjacente, visitados, fila) # chamada recursiva pegando o proximo vertice do set
        return visitados  # retorna a lista de visitados  
    
    def buscarPorValor(self, valor):# busca um valor no grafo usando a busca em profundidade O(|V|+|E|)
        for v in self.bfs(next(iter(self.getVertices())), visitados = [], fila = deque([])):
            if valor == v.getValor():
                return valor
        return None
    
    def existeCaminhoEuler(self):# checa a existencia de um caminho Euleriano no grafo
        pares = True
        impares = []
        for v in self.getVertices():
            if v.getGrau() % 2 != 0:
                pares = False
                impares.append(v)
        return pares or len(impares) == 2
    
    def removerAresta(self, origem, destino, peso = 1): # remove uma aresta do grafo
        try:
            verticeOrigem  = self.getVerticeByValor(origem)
            verticeDestino = self.getVerticeByValor(destino)
            if (verticeOrigem or verticeDestino) is None:  # existem os vertices de origem e destino?
                pass
            else:
                arestas_a_remover = []
                for aresta in self.getArestas():
                    if aresta.getVerticeOrigem() == verticeOrigem and aresta.getVerticeDestino() == verticeDestino and aresta.getPeso() == peso:
                        arestas_a_remover.append(aresta)
                [self.getArestas().remove(aresta) for aresta in arestas_a_remover]
        except AttributeError as error:
            print("Exceção: Nao ha no grafo, vertices de origem ou de destino com os valores informado!.")
            
    def removerVertice(self, valor):# remove um vertice do grafo
        try:
            vertice  = self.getVerticeByValor(valor)
            if vertice is None:  # existem os vertices de origem e destino?
                pass
            else:
                arestas_a_remover = []
                # arestas conectadas ao vertice precisam ser removidas antes
                for a in vertice.getArestasSaida():
                    arestas_a_remover.append(a)
                for a in vertice.getArestasEntrada():
                    arestas_a_remover.append(a)
                [self.removerAresta(a.getVerticeOrigem(), a.getVerticeDestino(), a.getPeso()) for a in arestas_a_remover]    
                self.__vertices.remove(vertice)
        except AttributeError as error:
            print(repr(error))
            print("Exceção: Nao ha no grafo, vertices de origem ou de destino com os valores informado.")
            
            
    def getMatrizAdjacencia(self):# retorna uma matriz de adjacencia como dataframe pandas
        import numpy as np
        import pandas as pd
        V = self.getVertices()
        E = self.getArestas()
        matriz = np.zeros((len(V), len(V)), dtype=object)
        columns = []
        [columns.append(v.getValor()) for v in V]
        index =[]
        [index.append(v.getValor()) for v in V]
        matrizDeAdjacencias = pd.DataFrame(matriz, columns = columns, index = index)
        for index, row in matrizDeAdjacencias.iterrows():
            for e in E:
                matrizDeAdjacencias.loc[e.getVerticeOrigem().getValor(), e.getVerticeDestino().getValor()] = e.getPeso()
        return matrizDeAdjacencias
    
    def getMatrizAdjacenciaAsArray(self):# converte a matriz de adjacencia pandas.dataframe para array
        return self.getMatrizAdjacencia().to_numpy()
    
    def getMatrizAdjacenciaAsDict(self):# converte a matriz de adjacencia pandas.dataframe para dicionario (lista de adjacencia)
        return self.getMatrizAdjacencia().to_dict('dict')

# Algoritmos de Busca

## Dijkstra

In [5]:
import heapq
class PriorityQueue:
    def __init__(self, elements = None):
      if elements == None:
        self.__elements = list()
      elif type(elements) == list:
        heapq.heapify(elements)
        self.__elements = elements
        
    def __str__(self):
        return str(self.__elements)
  
    def isEmpty(self):
        return len(self.__elements) == 0
  
    def push(self, element):
        heapq.heappush(self.__elements, element)

    def pop(self):
        return heapq.heappop(self.__elements)
    
    def remove(self, element):
        if not self.__elements.__contains__(element): return
        self.__elements.remove(element)
        heapq.heapify(self.__elements)

In [6]:
class Dijkstra:
    def __init__(self, verticeOrigem : Vertice):
        self.__verticeOrigem = verticeOrigem
        self.__verticeOrigem.setDistanciaMinima(0)
        filaVertice = PriorityQueue()
        filaVertice.push(self.__verticeOrigem)
        while(not filaVertice.isEmpty()):
            tempVertice = filaVertice.pop()
            for a in tempVertice.getArestasSaida():
                v = a.getVerticeDestino()
                distanciaMinima = tempVertice.getDistanciaMinima() + a.getPeso()
                if distanciaMinima < v.getDistanciaMinima():
                    filaVertice.remove(v)
                    v.setDistanciaMinima(distanciaMinima)
                    v.setVerticeAnterior(tempVertice)
                    filaVertice.push(v)

    def getMenorCaminho(self, verticeDestino : Vertice, grafo : Grafo):
        menorCaminho = []
        vertice = verticeDestino
        while(vertice != None):
            menorCaminho.append(vertice)
            vertice = vertice.getVerticeAnterior()
        menorCaminho.reverse()
        return menorCaminho

# Testes

## Teste com Dijkstra

In [7]:
vA = Vertice("A")
vB = Vertice("B")
vC = Vertice("C")
vD = Vertice("D")
vE = Vertice("E")
vF = Vertice("F")

a1 = Aresta(vA, vB, 2)
a2 = Aresta(vA, vC, 3)
a3 = Aresta(vB, vD, 4)
a4 = Aresta(vC, vE, 2)
a5 = Aresta(vD, vF, 1)
a6 = Aresta(vE, vF, 2)

G = Grafo()
G.setVertices({ vA, vB, vC, vD, vE, vF  })
G.setArestas({  a1, a2, a3, a4, a5 , a6 })

In [8]:
for a in G.getArestas():
    print(f"{a.getVerticeOrigem().getValor()} ---{a.getPeso()}--> {a.getVerticeDestino().getValor()}")

C ---2--> E
A ---3--> C
B ---4--> D
E ---2--> F
D ---1--> F
A ---2--> B


In [9]:
dijkstra = Dijkstra(vA)

textoMelhorCaminho = "Melhor caminho: "
resultadoMenorCaminho = dijkstra.getMenorCaminho(vF, G)
for i, v in enumerate(resultadoMenorCaminho):
    textoMelhorCaminho += f"{v.getValor()}"
    if i < len(resultadoMenorCaminho) - 1:
        textoMelhorCaminho += " --> "
print(textoMelhorCaminho)

Melhor caminho: A --> B --> D --> F
